# create image noise 

In [37]:
import numpy as np
import cv2
import torch
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt

# Set the dimensions of your image
width, height = 800, 600

# Generate random noise (you can adjust the intensity by changing the scale)
noise = np.random.normal(loc=0, scale=25, size=(height, width, 3)) # change the scale to reduce or inhance the noise

# Create an empty image and add the noise
noise_img = np.zeros((height, width, 3), dtype=np.uint8)
noise = noise.astype(np.uint8)  # Convert noise to uint8
noise_img = cv2.addWeighted(noise_img, 1.0, noise, 1.0, 0)

# # Save or display the noise image
# cv2.imwrite(r'C:\Users\USER\Desktop\siber_test\photos\noise_image.png', noise_img)

# origina photo

In [69]:
# Model
model = torch.hub.load("ultralytics/yolov5", "yolov5s")
#Google Drive link to the photo
image_url = "https://drive.google.com/uc?id=1wd7hb6b_6eK0p_DWJuhOQeUiRudtB4IP"

# Fetch image from the URL
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))

# # Show prediction on the original photo
# model(img).show()

# Get the results
results = model(img)

# Extract confidence values for the class "person" (class index 0)
person_confidences = results.xyxy[0][results.xyxy[0][:, 5] == 0, 4].cpu().numpy()

# Print confidence values for the class "person"
print("Confidence values for the class 'person':")
for confidence in person_confidences:
    print(confidence)
    
results.show()

# add noise 

In [68]:
# see definition in code above
original = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
noise = cv2.cvtColor(noise_img, cv2.COLOR_RGB2BGR)

# Ensure images are of the same size
noise = cv2.resize(noise, (original.shape[1], original.shape[0]))

# Define a range of noise levels
noise_levels = [0.1, 0.3, 0.5, 0.7, 1.0]
original_percentages = [1 - noise_level for noise_level in noise_levels]

# Lists to store data for plotting
noise_percentages = []
accuracies = []

for noise_level, original_percentage in zip(noise_levels, original_percentages):
    # Add the images together
    noiseImage = cv2.addWeighted(original, original_percentage, noise, noise_level, 0)
    
    # Convert the result back to PIL format for showing
    noiseImage_pil = Image.fromarray(cv2.cvtColor(noiseImage, cv2.COLOR_BGR2RGB))
    
    # Print the percentage of noise
    print(f"Percentage of noise: {noise_level * 100}%")
    
    # Get the results
    results = model(noiseImage_pil)

    # Extract confidence values for the class "person" (class index 0)
    person_confidences = results.xyxy[0][results.xyxy[0][:, 5] == 0, 4].cpu().numpy()
#     print (person_confidences)
    
    if len(person_confidences) == 0:
        accuracies.append(0)
        
    # Print confidence values for the class "person"
    for confidence in person_confidences:
        accuracies.append(confidence)
    
    # Store data for plotting
    noise_percentages.append(noise_level * 100)
    
    # Show prediction on the noisy photo
    results.show()

In [67]:
# Plotting the graph
plt.plot(noise_percentages, accuracies, marker='o')
plt.title('Effect of Noise on YOLOv5 Prediction Accuracy')
plt.xlabel('Percentage of Noise')
plt.ylabel('Prediction Accuracy')
plt.grid(True)
plt.show()